### 1. Importing the needed libraries
---
Run the following commands 
``` 
pip install pip install pyarabic
pip install nltk
pip install textblob
```

In [234]:
import pandas as pd
import pyarabic.araby as araby
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from textblob import TextBlob
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dalal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [235]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### 2. Read the Data 
--- 
It is required to first read the data and ensure that it has ben parsed correctly. the following code block will read and store the .csv file into a data frame that we will clean up accordingly. then we will perform some data assessment and check for quality issues.

In [236]:
dirty_tweets = pd.read_csv('Tweets\dirty_tweets_updated.csv')
display(dirty_tweets.head())
display(dirty_tweets.tail())


,index,ID,Tweet,Timestamp,Likes,Retweets,Length
0,0,1497300372569739264,قبيل الاحتفال بـ #اليوم_الدولي_للمرأة.. تستعد كوكبة من الفنانات الاستثنائيات وفرق موسيقية بقيادات نسائية لإبهار زوار إكسبو 2020 دبي\n\n#عينك_على_إكسبو \n@expo2020dubai #Expo2020\nhttps://t.co/3yfbIstF6Q,2022-02-25 20:00:01+00:00,1,0,198
1,1,1497299670522994691,#JIMMY_4040 \nلحظات نزول دابانغ سلمان خان على أرض المسرح في اكسبو 2020 وهو يقف على مجسم لجناحي صقر والتصفيق مستمر من قبل الجمهور\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubai https://t.co/gTBM7OL4Fv,2022-02-25 19:57:14+00:00,2,0,207
2,2,1497297255245754371,ليلة مميزة بإنتظارنا في @expo2020dubai يوم ٢٦ فبراير 😍\n\nألحانها شرقية 🎶 و نجومها إستثنائيين ✨\n\n#Watary | @ziadbourji | @josephattieh | @DalidaKhalil | #BashaarAlJawad | #Expo2020 | #Expo2020Dubai | #اكسبو2020 https://t.co/QtmvqEBOwZ,2022-02-25 19:47:38+00:00,0,0,232
3,3,1497295489481510914,@NancyAFashion أصدق اي حفلة احلى ولا اي فستان ؟؟؟\nهلأ اذا مستان انا بختارر #Expo2020 \nواذا حفلة فبختارر حفلة #Riyadh \nهو حفلات نانسي كلون حلوين يعني 😉💖😘💖😘💖😘\n@NancyAjram,2022-02-25 19:40:37+00:00,3,1,168
4,4,1497290589800480779,#JIMMY_4040 \nجنون ما بعده جنون دابانغ سلمان خان يستعرض أمام حشد كبير من معجبينه في اكسبو 2020\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubaigirls https://t.co/5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,index,ID,Tweet,Timestamp,Likes,Retweets,Length
14832,14832,1510291302646272001,نهنئ الأمة الإسلامية والعربية بقدوم شهر رمضان المبارك✨🌙\n\n@expo2020dubai \n\n#رمضان \n#إكسبو_دبي \n#اكتشف_أمة https://t.co/9MPjRlw2C2,2022-04-02 16:21:21+00:00,7,2,128
14833,14833,1510285937493741573,"زار وزير التجارة الخارجية الإماراتي ثاني بن أحمد الزيودي، جناح ""إسرائيل"" في إكسبو 2020 دبي.\nواطّلع الوزير على الابتكارات التي قامت #إسرائيل بتطويرها، وفق ما قالته وسائل الإعلام الإماراتية، التي تروّج للاحتلال على أنه دولةٌ عظيمة صاحبة تطوّرٍ واقتصاد، متناسيةً إجرامه بحق #فلسطين. https://t.co/GtOYGPpknQ",2022-04-02 16:00:01+00:00,0,0,303
14834,14834,1510284707237449734,تمنيت لو كان له عمل عن اجنحه اكسبو دبي ٢٠٢٠ ولكن اتوقع انه راح يقوم بعمل جبار في حال استضافت المملكة العربية السعودية اكسبو في ٢٠٣٠ باذن الله تعالى. https://t.co/vyy3kU5T6u,2022-04-02 15:55:08+00:00,12,0,172
14835,14835,1510283016433811467,#إكسبو_2020_دبي وأكثر من 24 مليون زيارة.\n\nضيوف من مختلف الثقافات والجنسيات احتفوا بالمعرض العالمي.. ونجوم عالميين في جميع المجالات. \n\nوبعد ختام المعرض الدولي .. إليكم أبرز نجوم الرياضة الذين زاروا إكسبو 2020 دبي \n\n#Expo2020 \n#وام_رياضة https://t.co/H6zIu8qyRi,2022-04-02 15:48:25+00:00,2,1,260
14836,14836,1510282533505847300,اكسبو دبي# تعالو شوفو اي ال حصل اخر يوم # كوكب دبي# https://t.co/rCIe0Vd205 عبر @YouTube,2022-04-02 15:46:30+00:00,0,0,88


In [237]:
print("Number of tweets and columns", dirty_tweets.shape)
print(dirty_tweets.columns)

Number of tweets and columns (14837, 7)
Index(['index', 'ID', 'Tweet', 'Timestamp', 'Likes', 'Retweets', 'Length'], dtype='object')


In [238]:
dirty_tweets.dtypes

index         int64
ID            int64
Tweet        object
Timestamp    object
Likes         int64
Retweets      int64
Length        int64
dtype: object

In [239]:
dirty_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14837 entries, 0 to 14836
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      14837 non-null  int64 
 1   ID         14837 non-null  int64 
 2   Tweet      14837 non-null  object
 3   Timestamp  14837 non-null  object
 4   Likes      14837 non-null  int64 
 5   Retweets   14837 non-null  int64 
 6   Length     14837 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 811.5+ KB


In [240]:
dirty_tweets.describe()

,index,ID,Likes,Retweets,Length
count,14837.00000,14837.00000,14837.00000,14837.00000,14837.00000
mean,7418.00000,1504239583734685440.00000,6.88124,2.49882,159.73364
std,4283.21731,5021255954899275.00000,55.14281,19.34930,79.13496
min,0.00000,1494304767958401024.00000,0.00000,0.00000,16.00000
25%,3709.00000,1499838035948716032.00000,0.00000,0.00000,94.00000
50%,7418.00000,1501482930727800832.00000,1.00000,0.00000,153.00000
75%,11127.00000,1509621734475022336.00000,3.00000,1.00000,220.00000
max,14836.00000,1511844576201515008.00000,2981.00000,764.00000,406.00000


### 3. Assesment
---
We will be assessing both duplicates and null vallues and based on our observation we will conduct the proper cleaning process
> We will find the duplicates of the tweets column only, since the other columns are gaurnteed to have duplicates and they are simply metadata. The function gets the duplicate rows without the first occurunce. we will then sum and check how much of our data is duplicated.

In [241]:
print("Number of null values is \n", dirty_tweets.isnull().sum())


Number of null values is 
 index        0
ID           0
Tweet        0
Timestamp    0
Likes        0
Retweets     0
Length       0
dtype: int64


In [242]:
# Issue 1 - Assess Duplicates
duplicate_tweets = dirty_tweets[dirty_tweets.duplicated(['Tweet'], keep='first')]
display(duplicate_tweets.tail())

print(f'{duplicate_tweets.shape[0]} duplicates')

,index,ID,Tweet,Timestamp,Likes,Retweets,Length
14832,14832,1510291302646272001,نهنئ الأمة الإسلامية والعربية بقدوم شهر رمضان المبارك✨🌙\n\n@expo2020dubai \n\n#رمضان \n#إكسبو_دبي \n#اكتشف_أمة https://t.co/9MPjRlw2C2,2022-04-02 16:21:21+00:00,7,2,128
14833,14833,1510285937493741573,"زار وزير التجارة الخارجية الإماراتي ثاني بن أحمد الزيودي، جناح ""إسرائيل"" في إكسبو 2020 دبي.\nواطّلع الوزير على الابتكارات التي قامت #إسرائيل بتطويرها، وفق ما قالته وسائل الإعلام الإماراتية، التي تروّج للاحتلال على أنه دولةٌ عظيمة صاحبة تطوّرٍ واقتصاد، متناسيةً إجرامه بحق #فلسطين. https://t.co/GtOYGPpknQ",2022-04-02 16:00:01+00:00,0,0,303
14834,14834,1510284707237449734,تمنيت لو كان له عمل عن اجنحه اكسبو دبي ٢٠٢٠ ولكن اتوقع انه راح يقوم بعمل جبار في حال استضافت المملكة العربية السعودية اكسبو في ٢٠٣٠ باذن الله تعالى. https://t.co/vyy3kU5T6u,2022-04-02 15:55:08+00:00,12,0,172
14835,14835,1510283016433811467,#إكسبو_2020_دبي وأكثر من 24 مليون زيارة.\n\nضيوف من مختلف الثقافات والجنسيات احتفوا بالمعرض العالمي.. ونجوم عالميين في جميع المجالات. \n\nوبعد ختام المعرض الدولي .. إليكم أبرز نجوم الرياضة الذين زاروا إكسبو 2020 دبي \n\n#Expo2020 \n#وام_رياضة https://t.co/H6zIu8qyRi,2022-04-02 15:48:25+00:00,2,1,260
14836,14836,1510282533505847300,اكسبو دبي# تعالو شوفو اي ال حصل اخر يوم # كوكب دبي# https://t.co/rCIe0Vd205 عبر @YouTube,2022-04-02 15:46:30+00:00,0,0,88


6317 duplicates


In [243]:
# Issue 1 - Remove duplicates
dirty_tweets = dirty_tweets.drop_duplicates(subset='Tweet', keep="first")
dirty_tweets = dirty_tweets.reset_index(drop=True)
dirty_tweets = dirty_tweets.drop(['index'], axis=1)

tweets_copy = dirty_tweets.copy()

In [244]:
display(dirty_tweets.head())
display(dirty_tweets.tail())
print("Length after removing duplicates", dirty_tweets.shape[0])

,ID,Tweet,Timestamp,Likes,Retweets,Length
0,1497300372569739264,قبيل الاحتفال بـ #اليوم_الدولي_للمرأة.. تستعد كوكبة من الفنانات الاستثنائيات وفرق موسيقية بقيادات نسائية لإبهار زوار إكسبو 2020 دبي\n\n#عينك_على_إكسبو \n@expo2020dubai #Expo2020\nhttps://t.co/3yfbIstF6Q,2022-02-25 20:00:01+00:00,1,0,198
1,1497299670522994691,#JIMMY_4040 \nلحظات نزول دابانغ سلمان خان على أرض المسرح في اكسبو 2020 وهو يقف على مجسم لجناحي صقر والتصفيق مستمر من قبل الجمهور\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubai https://t.co/gTBM7OL4Fv,2022-02-25 19:57:14+00:00,2,0,207
2,1497297255245754371,ليلة مميزة بإنتظارنا في @expo2020dubai يوم ٢٦ فبراير 😍\n\nألحانها شرقية 🎶 و نجومها إستثنائيين ✨\n\n#Watary | @ziadbourji | @josephattieh | @DalidaKhalil | #BashaarAlJawad | #Expo2020 | #Expo2020Dubai | #اكسبو2020 https://t.co/QtmvqEBOwZ,2022-02-25 19:47:38+00:00,0,0,232
3,1497295489481510914,@NancyAFashion أصدق اي حفلة احلى ولا اي فستان ؟؟؟\nهلأ اذا مستان انا بختارر #Expo2020 \nواذا حفلة فبختارر حفلة #Riyadh \nهو حفلات نانسي كلون حلوين يعني 😉💖😘💖😘💖😘\n@NancyAjram,2022-02-25 19:40:37+00:00,3,1,168
4,1497290589800480779,#JIMMY_4040 \nجنون ما بعده جنون دابانغ سلمان خان يستعرض أمام حشد كبير من معجبينه في اكسبو 2020\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubaigirls https://t.co/5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,ID,Tweet,Timestamp,Likes,Retweets,Length
8515,1510697936317030406,أغلق معرض #إكسبو_2020 في #الإمارات، الذي أجَّلته الجائحة، أبوابه بعد ثماني سنوات من الترقّب، وسبعة مليارات دولار من الاستثمارات، و240 مليون ساعة من العمل، وستة أشهر من الاحتفالات. لكن هل ترك الحدث الإرث الحقيقي الذي كان متوقعاً؟\n\nالإجابة عبر السلسلة..👇(1)\nhttps://t.co/F1WDvRlhIE,2022-04-03 19:17:10+00:00,9,1,279
8516,1510688181640962050,اختتام معرض #اكسبو_٢٠٢٠_دبي \nقبل يومين الخميس \nوكان لي فرصة زيارة المعرض قبل فترة سرني ما شاهدته من حسن تنظيم وتظاهُره عالمية ناجحة نبارك لدولة #الإمارات الشقيقة هذا النجاح المتفرد\nوبإذن الله نأمل استضافة معرض إكسبو الدولي في الرياض عام 2030 تحقيق ل..\n#رؤية_السعودية_2030 \n#تصويري https://t.co/dSOzxXl4TF,2022-04-03 18:38:24+00:00,1,0,304
8517,1510686349417984006,"هاشتاق عربي - بعد انتهائه..الإمارات تكشف عن عدد زوار ""إكسبو دبي"" https://t.co/398zbr1SIh https://t.co/uDDkr4IVp7",2022-04-03 18:31:07+00:00,0,0,112
8518,1510686265649340420,الحمدلله انجاز رائع \n\nشكراً #الامارات الحبيبه \n\nنبارك نجاح #اكسبو_٢٠٢٠_دبي ونجاح جناح #السعودية_في_إكسبو2020\n\nونؤكد نجاح الامارات نجاحنا https://t.co/g1XNuxvPlN,2022-04-03 18:30:47+00:00,0,2,160
8519,1510686095054356487,#إكسبو2020\nأبرز الأرقام القياسية التي حققها جناح السعودية في إكسبو 2020 دبي https://t.co/rHZA48l8DX,2022-04-03 18:30:06+00:00,0,0,100


Length after removing duplicates 8520


In [245]:
# Issue 2 - Removal of diacritics and arabic letters normalization
def remove_diacritics(tweet):
    # remove diacritical marks 
    tweet = araby.strip_diacritics(tweet)
    tweet = araby.strip_tashkeel(tweet)

    # normalize arabic letters
    tweet = araby.normalize_alef(tweet)
    tweet = araby.normalize_hamza(tweet)
    tweet = araby.normalize_ligature(tweet)

    # replace each repeated characters with one instance only
    tweet = re.sub(r'(.)\1+', r'\1', tweet) 
    return tweet

In [246]:
# Issue 3 - Removal of unrelated characters 
def remove_emojis_unrelated_chars(tweet):
    # replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)

    # remove everything except alphanumeric
    tweet = re.sub('[/\W+/g]', ' ',  tweet)

    # remove underscore
    tweet = re.sub('[_]', ' ',  tweet)
    return tweet

In [247]:
# Issue 4 - Split date and time into different columns 
dirty_tweets['Date'] = pd.to_datetime(dirty_tweets['Timestamp']).dt.date
dirty_tweets['Time'] = pd.to_datetime(dirty_tweets['Timestamp']).dt.time

In [248]:
# Issue 5 - Removal of links and non-arabic words
def remove_links_foreign_words(tweet):
    # convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', tweet)

    # remove english words
    tweet = re.sub(r'\s*[A-Za-z]+\b', '', tweet).rstrip()
    tweet = "".join([char for char in tweet if char not in string.ascii_letters]).strip()

    # remove new lines with white space
    tweet = tweet.replace('\n', ' ')
    tweet = tweet.replace('\t', ' ')

    return tweet

In [249]:
ar_stops = set(stopwords.words('arabic'))
stop_words = {"،","","ورحمه","وبركاته","عليكم","السلام","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل",
"أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك",
"أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ",
"إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ",
"اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان",
"الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين",
"اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي",
"ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما",
"بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم",
"تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول",
"حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا",
"ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا",
"شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم",
"عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو",
"فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب",
"كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى",
"لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ",
"لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا",
"مازال","مافتئ","مايو","متى","مثل","مذ","مساء","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ",
"منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا",
"هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ",
"هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت",
"ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}

# Issue 6 - Removal of stop-words
def remove_stop_words(tweet):
    zen = TextBlob(tweet)
    words = zen.words
    return " ".join([w for w in words if not w in ar_stops and not w in stop_words and len(w) >= 2])


In [250]:
# Issue 7 - Remove digits
def remove_numbers(tweet):
    # remove [0-9] 
    tweet = re.sub(r'[0-9]+', '', tweet)

    # remove [٠-٩] 
    tweet = re.sub(r'[٠-٩]+', '', tweet)
    return tweet


In [251]:
# Function to clean up and process the tweets - it will be called for every record in tweets column
def clean_up(tweet):
    tweet = remove_diacritics(tweet)
    tweet = remove_links_foreign_words(tweet)
    tweet = remove_stop_words(tweet)
    tweet = remove_emojis_unrelated_chars(tweet)
    tweet = remove_numbers(tweet)
    return tweet

In [252]:
dirty_tweets["Tweet"] = dirty_tweets["Tweet"].apply(lambda tweet: clean_up(tweet))

In [253]:
display(tweets_copy.head())
display(dirty_tweets.head())

display(tweets_copy.tail())
display(dirty_tweets.tail())


,ID,Tweet,Timestamp,Likes,Retweets,Length
0,1497300372569739264,قبيل الاحتفال بـ #اليوم_الدولي_للمرأة.. تستعد كوكبة من الفنانات الاستثنائيات وفرق موسيقية بقيادات نسائية لإبهار زوار إكسبو 2020 دبي\n\n#عينك_على_إكسبو \n@expo2020dubai #Expo2020\nhttps://t.co/3yfbIstF6Q,2022-02-25 20:00:01+00:00,1,0,198
1,1497299670522994691,#JIMMY_4040 \nلحظات نزول دابانغ سلمان خان على أرض المسرح في اكسبو 2020 وهو يقف على مجسم لجناحي صقر والتصفيق مستمر من قبل الجمهور\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubai https://t.co/gTBM7OL4Fv,2022-02-25 19:57:14+00:00,2,0,207
2,1497297255245754371,ليلة مميزة بإنتظارنا في @expo2020dubai يوم ٢٦ فبراير 😍\n\nألحانها شرقية 🎶 و نجومها إستثنائيين ✨\n\n#Watary | @ziadbourji | @josephattieh | @DalidaKhalil | #BashaarAlJawad | #Expo2020 | #Expo2020Dubai | #اكسبو2020 https://t.co/QtmvqEBOwZ,2022-02-25 19:47:38+00:00,0,0,232
3,1497295489481510914,@NancyAFashion أصدق اي حفلة احلى ولا اي فستان ؟؟؟\nهلأ اذا مستان انا بختارر #Expo2020 \nواذا حفلة فبختارر حفلة #Riyadh \nهو حفلات نانسي كلون حلوين يعني 😉💖😘💖😘💖😘\n@NancyAjram,2022-02-25 19:40:37+00:00,3,1,168
4,1497290589800480779,#JIMMY_4040 \nجنون ما بعده جنون دابانغ سلمان خان يستعرض أمام حشد كبير من معجبينه في اكسبو 2020\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubaigirls https://t.co/5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,ID,Tweet,Timestamp,Likes,Retweets,Length,Date,Time
0,1497300372569739264,قبيل الاحتفال بـ اليوم الدولي لمراة تستعد كوكبة الفنانات الاستثناءيات وفرق موسيقية بقيادات نساءية لابهار زوار اكسبو دبي عينك علا اكسبو,2022-02-25 20:00:01+00:00,1,0,198,2022-02-25,20:00:01
1,1497299670522994691,لحظات نزول دابانغ سلمان خان علا ارض المسرح اكسبو يقف علا مجسم لجناحي صقر والتصفيق مستمر الجمهور,2022-02-25 19:57:14+00:00,2,0,207,2022-02-25,19:57:14
2,1497297255245754371,ليلة ميزة بانتظارنا الحانها شرقية نجومها استثناءين اكسبو,2022-02-25 19:47:38+00:00,0,0,232,2022-02-25,19:47:38
3,1497295489481510914,اصدق حفلة احلا فستان مستان انا بختار واذا حفلة فبختار حفلة حفلات نانسي كلون حلوين يعني,2022-02-25 19:40:37+00:00,3,1,168,2022-02-25,19:40:37
4,1497290589800480779,جنون بعده جنون دابانغ سلمان خان يستعرض حشد كبير معجبينه اكسبو,2022-02-25 19:21:09+00:00,5,2,178,2022-02-25,19:21:09


,ID,Tweet,Timestamp,Likes,Retweets,Length
8515,1510697936317030406,أغلق معرض #إكسبو_2020 في #الإمارات، الذي أجَّلته الجائحة، أبوابه بعد ثماني سنوات من الترقّب، وسبعة مليارات دولار من الاستثمارات، و240 مليون ساعة من العمل، وستة أشهر من الاحتفالات. لكن هل ترك الحدث الإرث الحقيقي الذي كان متوقعاً؟\n\nالإجابة عبر السلسلة..👇(1)\nhttps://t.co/F1WDvRlhIE,2022-04-03 19:17:10+00:00,9,1,279
8516,1510688181640962050,اختتام معرض #اكسبو_٢٠٢٠_دبي \nقبل يومين الخميس \nوكان لي فرصة زيارة المعرض قبل فترة سرني ما شاهدته من حسن تنظيم وتظاهُره عالمية ناجحة نبارك لدولة #الإمارات الشقيقة هذا النجاح المتفرد\nوبإذن الله نأمل استضافة معرض إكسبو الدولي في الرياض عام 2030 تحقيق ل..\n#رؤية_السعودية_2030 \n#تصويري https://t.co/dSOzxXl4TF,2022-04-03 18:38:24+00:00,1,0,304
8517,1510686349417984006,"هاشتاق عربي - بعد انتهائه..الإمارات تكشف عن عدد زوار ""إكسبو دبي"" https://t.co/398zbr1SIh https://t.co/uDDkr4IVp7",2022-04-03 18:31:07+00:00,0,0,112
8518,1510686265649340420,الحمدلله انجاز رائع \n\nشكراً #الامارات الحبيبه \n\nنبارك نجاح #اكسبو_٢٠٢٠_دبي ونجاح جناح #السعودية_في_إكسبو2020\n\nونؤكد نجاح الامارات نجاحنا https://t.co/g1XNuxvPlN,2022-04-03 18:30:47+00:00,0,2,160
8519,1510686095054356487,#إكسبو2020\nأبرز الأرقام القياسية التي حققها جناح السعودية في إكسبو 2020 دبي https://t.co/rHZA48l8DX,2022-04-03 18:30:06+00:00,0,0,100


,ID,Tweet,Timestamp,Likes,Retweets,Length,Date,Time
8515,1510697936317030406,اغلق معرض اكسبو الامارات اجلته الجاءحة ابوابه الترقب وسبعة مليارات الاستثمارات و ساعة العمل وستة اشهر الاحتفالات الحدث الارث الحقيقي متوقعا الاجابة عبر السلسلة,2022-04-03 19:17:10+00:00,9,1,279,2022-04-03,19:17:10
8516,1510688181640962050,اختام معرض اكسبو دبي يومين الخميس فرصة المعرض فترة سرني شاهدته حسن تنظيم وتظاهره عالمية ناجحة نبارك لدولة الامارات الشقيقة النجاح المتفرد وباذن اله نامل استضافة معرض اكسبو الدولي الرياض تحقيق رءية السعودية تصويري,2022-04-03 18:38:24+00:00,1,0,304,2022-04-03,18:38:24
8517,1510686349417984006,هاشتاق عربي انتهاءه الامارات تكشف عد زوار اكسبو دبي,2022-04-03 18:31:07+00:00,0,0,112,2022-04-03,18:31:07
8518,1510686265649340420,الحمدله انجاز راءع شكرا الامارات الحبيبه نبارك نجاح اكسبو دبي ونجاح جناح السعودية في اكسبو ونءكد نجاح الامارات نجاحنا,2022-04-03 18:30:47+00:00,0,2,160,2022-04-03,18:30:47
8519,1510686095054356487,اكسبو ابرز الارقام القياسية حقها جناح السعودية اكسبو دبي,2022-04-03 18:30:06+00:00,0,0,100,2022-04-03,18:30:06


In [254]:
duplicate_tweets = dirty_tweets[dirty_tweets.duplicated( ['Tweet'], keep='first')]
display(duplicate_tweets.tail())

print(f'{duplicate_tweets.shape[0]} duplicates')

dirty_tweets = dirty_tweets.drop_duplicates(subset='Tweet', keep="first")

,ID,Tweet,Timestamp,Likes,Retweets,Length,Date,Time
8481,1510875774869454849,البيان الاماراتية اكسبو دبي انطلاقة عالمية لمستقبل واعد لكل دول العالم,2022-04-04 07:03:50+00:00,0,0,100,2022-04-04,07:03:50
8484,1510861261357162501,الرواد لعقارات توقع استمرار زخم اكسبو علا عقارات دبي,2022-04-04 06:06:09+00:00,0,0,78,2022-04-04,06:06:09
8489,1510851948152557571,اليوسف مشاركة سلطنة عمان اكسبو دبي ساهمت دعم وتمكين المءسات الصغيرة والمتوسطة جريدة الرءية سلطنة عمان,2022-04-04 05:29:09+00:00,4,2,141,2022-04-04,05:29:09
8496,1510753005058019328,اكسبو دبي اجمالي عد الزوار زوار جناح السعودية,2022-04-03 22:55:59+00:00,92,42,75,2022-04-03,22:55:59
8503,1510726989967085575,مليونا استخدموا وساءل النقل الجماعي اكسبو دبي,2022-04-03 21:12:37+00:00,0,0,104,2022-04-03,21:12:37


545 duplicates


### 4. Save the Cleaned Data
--- 
Finally we will save the data back as a .csv file.

In [255]:
cleaned_tweets = dirty_tweets.copy()

cleaned_tweets = cleaned_tweets.reset_index(drop=True)

cleaned_tweets.to_csv('Tweets\\final_tweets_cleaned.csv')
